In [3]:
import requests
import json
import time
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#API key
api_key = "RGAPI-60b42ec8-92d6-4081-89a3-200cc18cb78a"
gameName = "Slylio" #username
tagLine = "EUW" #suffixe

In [5]:
#Player unviveraly unique identifier
requestPUUID = requests.get(f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}?api_key={api_key}")
puuid = requestPUUID.json()["puuid"] 


In [6]:
#Match history from PUUID 
nb_of_games = 20
games = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count={nb_of_games}&api_key={api_key}")
print(type(games))

<class 'requests.models.Response'>


In [11]:
#Extract matchs data from match ids
matchs = games.json()
matchs_data=[]
data = pd.DataFrame()
cout=0
for match in matchs:
    cout+=1
    print("Match n°",cout,"/",nb_of_games,"\r",end="")
    match_data = requests.get(f"https://europe.api.riotgames.com/lol/match/v5/matches/{match}?api_key={api_key}")
    matchs_data.append(match_data.json())
    print(match_data.json())



{'metadata': {'dataVersion': '2', 'matchId': 'EUW1_6829650766', 'participants': ['E3PSjIMSVp50qcIR91o7aI0foHEX5OOa6aKwch2PNbhcLUUi-YmvGc5Bu2aOuxYPJUlRq3f0lmCjuQ', 'pzXARB1WlM6syJgR5uOcOu0J6traUgBsyfSwiir3doeSKcm6Yd1LxJPUyy5wvEuCx4dYzdLKtpwXjw', '1UsVQ8COiQLnydf0YKEB1zO_EaKBem2iFmdaqU8Drbp8-2TeVlUfJD83k2bbIPCll97SpoltC4WspQ', 'kxMfgguHdkjOOkvBACvpAzE1awFOOP-_iBWgeP3oK38DfOydohdf5V5Q1dey3BtglAhPTcezC2ulfQ', '_i-qLXY1fQV7hg4c3_eDBn3-VtfqvZRcZCDEfDZuowfZXkaqn2mGBmY9Md_6onk-3mzIWPRAozU04Q', 'NfM6m-oCAqosvK1G6s-0v0Pg2ugRjLFwugacU3UEZVC2AWgYl9WZJWwIzJUgQGXLQUjoNX3Ifa1oYA', 'nICwUFHOH_TsQczqTIDUQioPzLXiN3fytgCmRxKhbq0puAX8nqzIEWAsPLP2prR7H3bQF3DOvJ4iyg', 'GtZ2FohJ_4_U92nVw7DQDQ5BWPbdRGOm9GPXf9iBzbLqnu2_3NIpNxR278bkP3FvshYfkj9hnmUu4A', 'waZHUOtKUzaRIVAEgEuUhxqkq9B31Sh4aF6iI9ui4mCG194msLN95-FgQd6BVclY2_zZN7T97BndAQ', 'L4Eay6JzvgG3RMFMVIzYHz4_2gjcF6qW4FYvkLKNlnnGO3fDiQl40eaEV5JEOxlfScbX40oMoeXAzQ']}, 'info': {'endOfGameResult': 'GameComplete', 'gameCreation': 1708778285400, 'gameDuration': 1833, 

In [57]:
#ENCODER One hot encoded
#sparse output = False
#set_output(transform="pandas")encoded
#sparse output = False
#set_output(transform="pandas")

                                             metadata  \
0   {'dataVersion': '2', 'matchId': 'EUW1_68202692...   
1   {'dataVersion': '2', 'matchId': 'EUW1_68182438...   
2   {'dataVersion': '2', 'matchId': 'EUW1_68178762...   
3   {'dataVersion': '2', 'matchId': 'EUW1_68178322...   
4   {'dataVersion': '2', 'matchId': 'EUW1_68177911...   
5   {'dataVersion': '2', 'matchId': 'EUW1_68177510...   
6   {'dataVersion': '2', 'matchId': 'EUW1_68177361...   
7   {'dataVersion': '2', 'matchId': 'EUW1_68177048...   
8   {'dataVersion': '2', 'matchId': 'EUW1_68176854...   
9   {'dataVersion': '2', 'matchId': 'EUW1_68176548...   
10  {'dataVersion': '2', 'matchId': 'EUW1_68156073...   
11  {'dataVersion': '2', 'matchId': 'EUW1_68155531...   
12  {'dataVersion': '2', 'matchId': 'EUW1_68154798...   
13  {'dataVersion': '2', 'matchId': 'EUW1_68152897...   
14  {'dataVersion': '2', 'matchId': 'EUW1_68129602...   
15  {'dataVersion': '2', 'matchId': 'EUW1_68128752...   
16  {'dataVersion': '2', 'match

In [ ]:
#demi CLASSIFIER